In [1]:
import pandas as pd
import os
import re

In [2]:
train_df = pd.read_csv('../data_processed/train_data.csv')

In [3]:
train_df.head()

,init_text,text,label
0,(Image credit: Image Credit: Methodshop / Pixa...,image credit image credit methodshop pixabay d...,honeypot
1,"Cyber-attacks in education are not new, but am...",cyber-attack education new amid sharp increase...,application_control
2,McAfee and Ingram Micro have announced that th...,mcafee ingram micro announce full mvision port...,SASE
3,Join GamesBeat Summit 2021 this April 28-29. R...,join gamesbeat summit YEARNUM april register f...,malware_detect
4,"CAMPBELL, Calif., Jan. 14, 2021 /PRNewswire/ -...",campbell calif jan YEARNUM prnewswire highligh...,SASE


In [4]:
train_df['label'] = train_df['label'].apply(lambda x: [x])

In [9]:
# check class distribution
dict_l = {}
for y_t in train_df['label'].values.tolist():
    if not str(y_t) in dict_l.keys():
        dict_l[str(y_t)] = 1
    else:
        dict_l[str(y_t)] += 1

In [10]:
dict_l

{"['honeypot']": 1623,
 "['application_control']": 1567,
 "['SASE']": 1584,
 "['malware_detect']": 1610,
 "['ransomware_detect']": 1557,
 "['anti-counterfeit']": 1610,
 "['NDR']": 1596,
 "['sandbox']": 1580,
 "['CASB']": 1622,
 "['EDR']": 1654,
 "['atm_security']": 1604,
 "['unwanted_programs']": 1583,
 "['NGFW']": 1554,
 "['spyware_detect']": 1606,
 "['MDR']": 1577,
 "['industrial_iot_security']": 1580,
 "['parental_control']": 1588,
 "['system_tweak']": 1580,
 "['SOAR']": 1585,
 "['SIEM']": 1578}

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

In [6]:
mlb = MultiLabelBinarizer()
mlb.fit_transform(train_df['label'].values.tolist())

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [7]:
mlb.classes_

array(['CASB', 'EDR', 'MDR', 'NDR', 'NGFW', 'SASE', 'SIEM', 'SOAR',
       'anti-counterfeit', 'application_control', 'atm_security',
       'honeypot', 'industrial_iot_security', 'malware_detect',
       'parental_control', 'ransomware_detect', 'sandbox',
       'spyware_detect', 'system_tweak', 'unwanted_programs'],
      dtype=object)

In [5]:
X_train = train_df['text'].values.tolist()

In [6]:
y_train = train_df['label'].values.tolist()

In [8]:
y_train = mlb.transform(y_train)

In [10]:
y_train[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

In [16]:
TV = TfidfVectorizer(min_df=10)   
X = TV.fit_transform(X_train).toarray()

In [17]:
TV.get_feature_names_out()[-50:]

array(['zindel', 'zingbox', 'zion', 'zionism', 'zionist', 'zip', 'zipper',
       'zire', 'ziring', 'ziser', 'ziv', 'zix', 'zk', 'zloader', 'znga',
       'zoe', 'zohar', 'zoho', 'zombie', 'zonal', 'zone', 'zonealarm',
       'zoning', 'zoo', 'zoom', 'zoonotic', 'zpa', 'zpe', 'zr', 'zs',
       'zscaler', 'zt', 'zta', 'ztap', 'zte', 'ztedge', 'zti', 'ztn',
       'ztna', 'ztp', 'zuck', 'zuckerberg', 'zug', 'zuma', 'zuorat',
       'zurcher', 'zurich', 'zvi', 'zylo', 'zynga'], dtype=object)

In [18]:
print(np.shape(X))
print(np.shape(y_train))

(31838, 26887)
(31838, 20)


Training logistic regression with multi-label output

In [19]:
baseline_clf = MultiOutputClassifier(estimator=LogisticRegression(n_jobs=2, verbose=1)).fit(X, y_train)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        26888     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.20684D+04    |proj g|=  1.42970D+04


 This problem is unconstrained.



At iterate   50    f=  2.98699D+03    |proj g|=  1.88555D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
26888     76     86      1     0     0   1.119D-02   2.987D+03
  F =   2986.9486089520860     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:   37.7s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


...

[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:   30.8s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        26888     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.20684D+04    |proj g|=  1.43360D+04


 This problem is unconstrained.



At iterate   50    f=  2.97212D+03    |proj g|=  4.90664D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
26888     63     70      1     0     0   1.222D-02   2.972D+03
  F =   2972.1187042021402     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:   30.6s finished


In [20]:
import pickle

In [21]:
# save the model to disk
filename = '../models/logistic_reg.sav'
pickle.dump(baseline_clf, open(filename, 'wb'))

In [22]:
# save tf-idf vectorizer
filename = '../models/tf_idf_vec.sav'
pickle.dump(TV, open(filename, 'wb'))

In [24]:
# save mlb
with open('../models/mlb.pkl', 'wb') as f:
    pickle.dump(mlb, f)

Training Desicion Tree with multi-label output

In [25]:
from sklearn import tree

In [29]:
dec_tree_clf = MultiOutputClassifier(estimator=tree.DecisionTreeClassifier()).fit(X, y_train)

In [30]:
# save the model to disk
filename = '../models/dec_tree.sav'
pickle.dump(dec_tree_clf, open(filename, 'wb'))

Attempts to get BERT vector representations (couldn't do it because do not have enough memory)

In [ ]:
import numpy as np
import torch
import transformers as ppb
from transformers import AutoTokenizer, DistilBertModel
from torch import cuda

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model = DistilBertModel.from_pretrained(model_name)
model.to(device)

Kernel умирает из-за нехватки памяти, так как пытаемся положить полученные вектора на cpu, чтобы sklearn модели могли их использовать (т.к они не могут работать на gpu)

In [ ]:
encoded_list = []
batch_size = 100 
sentences = train_df['init_text'].values.tolist()
with torch.no_grad():
    for idx in range(0, len(sentences), batch_size):
        batch = sentences[idx : min(len(sentences), idx+batch_size)]

        encoded = tokenizer.batch_encode_plus(batch,max_length=128, padding='max_length', truncation=True)

        encoded = {key:torch.LongTensor(value) for key, value in encoded.items()}
        ids = encoded['input_ids'].to(device)
        mask = encoded['attention_mask'].to(device)

        outputs = model(input_ids=ids, attention_mask=mask)[0]
        encoded_list.extend(outputs.cpu().detach().numpy().tolist())  

Дальше планировалось повторить эксперименты с логистической регрессией и decision tree и сравнить результаты работы с применением разных методов векторизации текста.